![](/files/images/dbxatscale.png)

#Overview
Now that we have our model training on a schedule, we want to use it on a schedule. In this notebook we will classify with our model and write our classifications to a new Databricks table. First we need to establish our AtScale connection, so we will use XX_Establish_AtScale_Connection to connect.

In [0]:
%run ./XX_Establish_AtScale_Connection

Next we need to read our model so we can use it for classification. MLFlow makes this process super easy, all we need to do is specify the name, a path to our model, and load it using mlflow.pyfunc.load_model. Then we can print our model out to make sure we have the successfully loaded our model

In [0]:
import mlflow
model_name = "Databricks_AI_Link_M5_Demo"

model_uri=f"models:/{model_name}/latest"
model = mlflow.pyfunc.load_model(model_uri=model_uri)
print(model)

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: bdef39ee21b04ddbab4cf667b270e4f3



Now let's load our data that we are going to classify on. We will do this by generating a spark dataframe from our query which was initialized in our XX_Establish_AtScale_Connection, then we will convert the spark dataframe to pandas and use pandas' drop function to drop out our item column. In the realistic use case of this model we don't know what item the row represents, so let's drop out item.

In [0]:
df = spark.sql(query).toPandas().drop(["item"], axis = 1)

Now predicting with our model is as easy as setting the value of a new column equal to the output of model.predict for our data

In [0]:
df["predicted_class"] = model.predict(df)

[LightGBM] [Warning] lambda_l2 is set=0.176191006422464, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.176191006422464
[LightGBM] [Warning] lambda_l1 is set=0.2347409701792684, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2347409701792684


In [0]:
df.head()

,average_sales,average_units_sold,max_sales,max_units_sold,population_variance_sales,population_variance_units_sold,sample_standard_deviation_sales,sample_standard_deviation_units_sold,sample_variance_units_sold,total_categories,total_departments,total_items,total_sales,total_states,total_stores,total_transactions,total_units_sold,day_over_day_units_sold,previous_days_units_sold,total_sales_30_prd_mv_avg,total_units_sold_28_day_max,total_units_sold_30_prd_mv_avg,date,predicted_class
0,1.960,7.5,1.96,19.0,-2.131628e-15,36.45,0.000000,6.363961,40.500000,1.0,1.0,1.0,19.60,3.0,10.0,10.0,75.0,0.0,NaN,19.60,75.0,75,2015-01-01,1
1,0.980,8.3,0.98,23.0,-5.329071e-16,35.01,0.000000,6.236986,38.900000,1.0,1.0,1.0,9.80,3.0,10.0,10.0,83.0,0.0,NaN,9.80,83.0,83,2015-01-01,1
2,3.160,4.3,3.28,10.0,3.360000e-02,11.01,0.193218,3.497618,12.233333,1.0,1.0,1.0,31.60,3.0,10.0,10.0,43.0,0.0,NaN,31.60,43.0,43,2015-01-01,1
3,3.160,4.4,3.28,10.0,3.360000e-02,10.24,0.193218,3.373096,11.377778,1.0,1.0,1.0,31.60,3.0,10.0,10.0,44.0,0.0,NaN,31.60,44.0,44,2015-01-01,1
4,1.379,0.0,1.48,0.0,2.112900e-02,0.00,0.153221,0.000000,0.000000,1.0,1.0,1.0,13.79,3.0,10.0,10.0,0.0,0.0,NaN,13.79,0.0,0,2015-01-01,1


Now we want to convert our resulting dataframe back to spark so we can write our dataframe to a table using sparks write save as table function. We will construct our spark dataframe called classifed_table using a spark CreateDataFrame call. We needed to move our dataframe to a pandas dataframe becuase models can not take in a spark dataframe.

In [0]:
classifed_table = spark.createDataFrame(df)

We will write this table to a table in atscale_sample_data under the ai_link_ml_testing schema. If we wanted to write this table to any other data warehouse AI Link enables us to do that. Say for instance, your buisness team operates off of Snowflake but your data engineers work in Databricks, AI Link facilitates the movement of data across warehouses to ensure that your one source of truth it available to all. Lastly we have to set our mode to overwrite so we ensure we have the most up to date verison of our classifications

In [0]:
classifed_table.write.saveAsTable("atscale_sample_data.ai_link_ml_testing.m5_model_item_class_predictions", mode = "overwrite")

### Create Webhook 
This will let our developers know when their model has been used to classify. To set up our webhooks we need to again run our set up notebook.

In [0]:
%run ./06_Webhook_Set_Up

In [0]:
now = datetime.now()

Webhook Set Up Complete.


In [0]:
#format success message
target_col = "item type"
messages = ["V--------------PREDICTION COMPLETE----------------V"]
messages += ["MODEL NAME: " + model_name + " LATEST VERSION"] 
messages += ["DATE: " + str(now)]
messages += ["PREDICTION TARGET: " + target_col]
messages += ["MODEL URI: " + model_uri]
messages += ["*******************************************"]
messages += [" "]
messages += [" "]
messages += [" "]

In [0]:
send_slack_messages(slack_url, channel, messages)

![](/files/images/atscale_logo.png)